In [1]:

pip install --upgrade pip
pip install farm-haystack[colab,preprocessing,elasticsearch]

Couldn't find program: 'bash'


In [2]:
from haystack.telemetry import tutorial_running

tutorial_running(3)

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [1]:
from elasticsearch import Elasticsearch

In [10]:
from elasticsearch import Elasticsearch

# Replace with your Elasticsearch host and port
es = Elasticsearch(hosts=["localhost:9200"])

# Test the connection
if es.ping():
    print("Connection to Elasticsearch successful")
else:
    
    print("Connection to Elasticsearch failed")

Connection to Elasticsearch successful


In [ ]:
import time
time.sleep(30)

In [2]:
from haystack.utils import launch_es
launch_es()

Tried to start Elasticsearch through Docker but this failed. It is likely that there is already an existing Elasticsearch instance running. 


In [3]:
import os
from haystack.document_stores import ElasticsearchDocumentStore

# Get the host where Elasticsearch is running, default to localhost
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")

document_store = ElasticsearchDocumentStore(
    host=host,
    username="",
    password="",
    index="document"
)

In [4]:
doc_dir = "C:/Users/User/Documents/Study/Degree/Year 3/Sem 1/Natural Language Processing/project/cleaned data"  # Specify the output directory path


In [ ]:
import os

if os.path.exists(doc_dir):
    print("Directory exists")
else:
    print("Directory does not exist")

# Fetch text files from the doc_dir directory
text_files = []
for file_name in os.listdir(doc_dir):
    if file_name.endswith(".txt"):
        file_path = os.path.join(doc_dir, file_name)
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
            text_files.append(content)

# Print the fetched text files
for content in text_files:
    print(content)

fine tuning

In [38]:
from haystack.telemetry import tutorial_running

tutorial_running(2)

In [39]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

In [40]:
from haystack.nodes import FARMReader

In [ ]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
# reader.save(directory="/content/drive/MyDrive/NAT LANGUAGE GROUP 22/100 Articles - Food/my_model")

In [6]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=500,
    split_overlap=20,
    split_respect_sentence_boundary=True,
)

In [7]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])


In [8]:
files_to_index = [doc_dir + "/" + f for f in os.listdir(doc_dir)]
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/200 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/200 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.
Document ae36107c34446882d475a324daf29408 is 10567 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document 76c86180f7bdedaaf20334fcf1c7bd76 is 13454 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document e22456e44a6b7e6759b6fc5f4f22ed9b is 20780 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to prevent issues at query time.
Document af9b88568fdbe7913088b2678c1f2773 is 11478 characters long after preprocessing, where the maximum length should be 10000. Something might be wrong with the splitting, check the document affected to preve

{'documents': [<Document: {'content': 'Nasi yang jadi makan mereka bungkus dengan daun pisang atau itip panggil Nuba Laya Lup adalah makan tradisional iaitu buat daripada beras pulut yang adun sama gula merah minyak dan garam Dapat juga kuih tradisional seperti Param dan Nunu Daging atau ikan yang telah peram dengan garam serta simpan di dalam buluh lama lebih bulan panggil telu serta rasa akan jadi masam masam masin sangat gari suku kaum Lun Bawang Begitu juga dengan salai narar Mereka juga dapat sumber garam dari telaga air masin yang lebih nali bagai lubang maingaram bukit kandung khaisat yang tinggi antara tinggi kandung iodin tingkat tahap intelektual utama di kalang anak kecil natrium kalsium kalium zat besi fosforusmagnesium mangan kuprum dan juga zink Lun Bawang bawa maksud orang di kawasan dalam people of the interior Lun maksud orang manakala Bawang bawa maksud ulu Cara seluruh Lun Bawang erti orang ulu Pada amnya daripada 25 atau lebih suku kaum yang dapat di Sarawak yang ra

In [9]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [10]:
from haystack.nodes import FARMReader

#reader = FARMReader(model_name_or_path="malay-huggingface/bert-tiny-bahasa-cased", use_gpu=True)
reader = FARMReader(model_name_or_path="C:/Users/User/Documents/Study/Degree/Year 3/Sem 1/Natural Language Processing/project/my_model", use_gpu=True)

Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": true, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}


In [11]:
from haystack import Pipeline

querying_pipeline = Pipeline()
querying_pipeline.add_node(component=retriever, name="Retriever", inputs=["Query"])
querying_pipeline.add_node(component=reader, name="Reader", inputs=["Retriever"])

In [12]:
prediction = querying_pipeline.run(
    query="Apakah makanan tradisional melayu?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 1}
    }
)

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]

In [ ]:
"""# just run this code if u have error on previous code, if not please dont run it

!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz

import os
from subprocess import Popen, PIPE, STDOUT

# Start Elasticsearch
es_process = Popen(['elasticsearch-7.9.2/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))"""


"# just run this code if u have error on previous code, if not please dont run it\n\n!wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q\n!tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz\n\nimport os\nfrom subprocess import Popen, PIPE, STDOUT\n\n# Start Elasticsearch\nes_process = Popen(['elasticsearch-7.9.2/bin/elasticsearch'], stdout=PIPE, stderr=STDOUT, preexec_fn=lambda: os.setuid(1))"

In [13]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Ketupat nasi', 'type': 'extractive', 'score': 0.01772889494895935, 'context': ' ketupat yang kenal di Malaysia iaitu ketupat nasi dan ketupat Palas Ketupat nasi buat daripada daun kelapa dan beras nasi manakala ketupat palas pula', 'offsets_in_document': [{'start': 1373, 'end': 1385}], 'offsets_in_context': [{'start': 69, 'end': 81}], 'document_ids': ['be6bbb3036b1fcae468c09f5248084e6'], 'meta': {'_split_id': 0, '_split_overlap': []}}>],
 'documents': [<Document: {'content': 'Makan Tradisional Ikut adalah senarai makan tradisional melayu yang popular di Malaysia Makan Tradisional Melayu Lemang Apakah makan tradisional melayu paling popular Jawapannya adalah Lemang Lemang adalah makan tradisional Melayu yang sangat popular dan paling dapat sambut ramai adalah lemang Resep lemang adalah diri daripada beras pulut santan dan garam Bahanbahan ini akan masak perlahanlahan di dalam buluh yang di dalam sudah balut dengan daun pisang Ketupat sedap makan sama ren

In [49]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)

'Query: Apakah makanan tradisional melayu?'
'Answers:'
[   {   'answer': 'Ketupat nasi',
        'context': ' ketupat yang kenal di Malaysia iaitu ketupat nasi dan '
                   'ketupat Palas Ketupat nasi buat daripada daun kelapa dan '
                   'beras nasi manakala ketupat palas pula'}]


In [50]:
prediction = querying_pipeline.run(
    query="Apakah nasi lemak?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 1}
    }
)

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

In [51]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Nasi lemak rujuk kepada nasi yang masak dengan guna santan kelapa bagi tambah rasa lemak Kadangkala daun pandan masuk masa nasi lemak masak bagi tambah aroma Nasi lemak biasa hidang dengan telur timun ikan bilis goreng dan sambal Tetapi kini nasi lemak jual dengan pelbagai lauk yang masuk tetapi tidak had kepada daging ayam sotong udang limpa dan hati lembu Laksa rupa jenis makan kuah seperti mi yang popular di Malaysia dan Singapura Ia kata asal dari orang Cina', 'type': 'extractive', 'score': 0.016754213720560074, 'context': 'Nasi lemak rujuk kepada nasi yang masak dengan guna santan kelapa bagi tambah rasa lemak Kadangkala daun pandan masuk masa nasi lemak masak bagi tambah aroma Nasi lemak biasa hidang dengan telur timun ikan bilis goreng dan sambal Tetapi kini nasi lemak jual dengan pelbagai lauk yang masuk tetapi tidak had kepada daging ayam sotong udang limpa dan hati lembu Laksa rupa jenis makan kuah seperti mi yang popular di Malaysia dan Singa

In [52]:
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)

'Query: Apakah nasi lemak?'
'Answers:'
[   {   'answer': 'Nasi lemak rujuk kepada nasi yang masak dengan guna santan '
                  'kelapa bagi tambah rasa lemak Kadangkala daun pandan masuk '
                  'masa nasi lemak masak bagi tambah aroma Nasi lemak biasa '
                  'hidang dengan telur timun ikan bilis goreng dan sambal '
                  'Tetapi kini nasi lemak jual dengan pelbagai lauk yang masuk '
                  'tetapi tidak had kepada daging ayam sotong udang limpa dan '
                  'hati lembu Laksa rupa jenis makan kuah seperti mi yang '
                  'popular di Malaysia dan Singapura Ia kata asal dari orang '
                  'Cina',
        'context': 'Nasi lemak rujuk kepada nasi yang masak dengan guna santan '
                   'kelapa bagi tambah rasa lemak Kadangkala daun pandan masuk '
                   'masa nasi lemak masak bagi tambah aroma Nasi lemak biasa '
                   'hidang dengan telur timun ikan bilis goreng 

In [94]:
prediction = querying_pipeline.run(
    query="Apakah maksud Ambuyat?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 1}
    }
)

Inferencing Samples:   0%|          | 0/2 [00:00<?, ? Batches/s]

In [95]:
from pprint import pprint

pprint(prediction)

{'answers': [<Answer {'answer': 'Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak dan juga Brunei', 'type': 'extractive', 'score': 0.02122381702065468, 'context': 'Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak dan juga Brunei Darussalam Sabah ya', 'offsets_in_document': [{'start': 0, 'end': 130}], 'offsets_in_context': [{'start': 0, 'end': 130}], 'document_ids': ['dfaa721707e6dad88ff86288c540619c'], 'meta': {'_split_id': 0, '_split_overlap': []}}>],
 'documents': [<Document: {'content': 'Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak dan juga Brunei Darussalam Sabah yang kenal dengan pelbagai bangsa seperti Kadazan Murut Dusun dan Bajau juga pati makan ini Ambuyat rupa saji utama bagi kaum Bisaya lepas nasi Makan ini hidang bagai menu hari dan rupa hidang yang wajib masa harihari istime

In [122]:
from haystack.utils import print_answers


print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium` and `all`
)

answer = prediction['answers'][0].answer
print(answer)

'Query: Apakah maksud Ambuyat?'
'Answers:'
[   {   'answer': 'Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah '
                  'dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak '
                  'dan juga Brunei',
        'context': 'Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah '
                   'dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak '
                   'dan juga Brunei Darussalam Sabah ya'}]
Ambuyat rupa makan tradisional bagi kaum Bisaya di Sabah dan Brunei dan kenal di Sabah Wilayah Sekutu Labu Sarawak dan juga Brunei


In [123]:
import tkinter as tk

def send_message():
    user_input = input_box.get().strip() 
    if not user_input:
        return  
        
    display_message(user_input, "user")  
    response = run_query(user_input)
    display_message(response, "bot")  
    input_box.delete(0, tk.END)  

def display_message(message, sender):
    chat_log.configure(state='normal')  
    chat_log.insert(tk.END, f"{sender}: {message}\n")  
    chat_log.configure(state='disabled') 
    chat_log.see(tk.END)  

def run_query(query):
    # Run the query and get the prediction
    prediction = querying_pipeline.run(
        query="Apakah makanan tradisional melayu?",
        params={
            "Retriever": {"top_k": 10},
            "Reader": {"top_k": 1}
        }
    )
    return prediction
    
window = tk.Tk()
window.title("Chatbot")
window.geometry("500x500")

label = tk.Label(window, text="Makanan Malaysia", font=("Helvetica", 16, "bold"))
label.pack(pady=10)

chat_log = tk.Text(window, height=20, width=60)
chat_log.configure(state='disabled') 
chat_log.pack(padx=10, pady=10)

input_box = tk.Entry(window, width=60)
input_box.pack(padx=10, pady=10)

send_button = tk.Button(window, text="Send", command=send_message)
send_button.pack(pady=10)

window.mainloop()

Inferencing Samples:   0%|          | 0/3 [00:00<?, ? Batches/s]